In [ ]:
!pip install torch numpy google.colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

# gpu available
if torch.cuda.is_available():
    print("GPU is available", torch.cuda.get_device_name(0))
else:
    raise RuntimeError("GPU is not available")

GPU is available Tesla T4


In [ ]:
#Hyperparameters:
batch_size = 16  # Number of independent sequences processed in parallel
block_size = 128  # Maximum context length for predictions
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 488
n_head = 7
n_layer = 6
dropout = 0.1
train = 0.9
#torch.manual_seed(1337) #Doing seeded tests for the sake of testing

In [ ]:
# Load the dataset
file_path = '/content/alllines.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    print("Length of dataset in characters is:", len(text))
except FileNotFoundError:
    raise FileNotFoundError(f"Error: File missing {file_path}")
except Exception as e:
    raise RuntimeError(f"Error {e}")

Length of dataset in characters is: 4583798


In [ ]:
# Character mappings
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(train * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)  # pray data is moved to T4
    return x, y

In [ ]:
#Classes:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
# custom LayerNorm normalizes input by mean and standard deviation, then scales and shifts with learnable parameters - I know that
class LayerNorm(nn.Module):
    def __init__(self, n_embd, eps=1e-6):
        super(LayerNorm, self).__init__()

        self.gamma = nn.Parameter(torch.ones(n_embd))
        self.beta = nn.Parameter(torch.zeros(n_embd))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)

        normalized = (x-mean)/torch.sqrt(std+self.eps) #standard deviation is already the square root of the varianec but using the sqrt gives better restults. I don't know why but it works ish
        return self.gamma * normalized + self.beta


In [ ]:
# Define model components - a single attention head performing scaled dot-product attention with masking, dropout, and learned Query/Key/Value projections
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        sq_n_embd = np.sqrt(1 / n_embd)
        self.W_q = nn.Parameter(torch.rand(n_embd, head_size) * 2 * sq_n_embd - sq_n_embd)
        self.W_k = nn.Parameter(torch.rand(n_embd, head_size) * 2 * sq_n_embd - sq_n_embd)
        self.W_v = nn.Parameter(torch.rand(n_embd, head_size) * 2 * sq_n_embd - sq_n_embd)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = torch.matmul(x, self.W_k)
        q = torch.matmul(x, self.W_q)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = torch.matmul(x, self.W_v)
        out = wei @ v
        return out


In [ ]:
#implement multi-head attention by combining outputs from multiple atention heads and projecting them into the model's embedding space
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
# two-layer feedforward network with ReLU activation
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
#performs attention (ln1), adds output, applies feedforward (ln2), adds result - basic transformer block
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = LayerNorm(n_embd)
        self.ln2 = LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [ ]:
# maps token and position embeddings through transformer blocks, applies layer normalization,
# predicts logits via a linear head, optimizes with cross-entropy, and generates text iteratively using autoregressive decoding

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Parameter(torch.zeros(block_size, n_embd))
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table[:T, :]
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            return logits, None
        else:
            logits = logits.view(B * T, -1)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
# Initialize model
model = GPTLanguageModel().to(device)

# GPU usage
print(f"Model is on device: {next(model.parameters()).device}")
print(f"Number of model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M parameters")

# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Track perplexity
train_perplexity = []
val_perplexity = []

# Training loop with perplexity
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_loss = losses['train']
        val_loss = losses['val']
        train_perplexity = torch.exp(torch.tensor(train_loss)).item()
        val_perplexity = torch.exp(torch.tensor(val_loss)).item()
        print(f"Step {iter}: Train Loss {train_loss:.4f}, Val Loss {val_loss:.4f}, Val Perplexity: {val_perplexity:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate text
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


Model is on device: cuda:0
Number of model parameters: 17.26M parameters


<ipython-input-21-a409509d65b2>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_perplexity = torch.exp(torch.tensor(train_loss)).item()
<ipython-input-21-a409509d65b2>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_perplexity = torch.exp(torch.tensor(val_loss)).item()


Step 0: Train Loss 4.4381, Val Loss 4.4410, Val Perplexity: 84.8602
Step 100: Train Loss 2.3746, Val Loss 2.3825, Val Perplexity: 10.8315
Step 200: Train Loss 2.3260, Val Loss 2.3298, Val Perplexity: 10.2759
Step 300: Train Loss 2.3099, Val Loss 2.3186, Val Perplexity: 10.1617
Step 400: Train Loss 2.2806, Val Loss 2.2876, Val Perplexity: 9.8511
Step 500: Train Loss 2.2236, Val Loss 2.2300, Val Perplexity: 9.2994
Step 600: Train Loss 2.0867, Val Loss 2.0887, Val Perplexity: 8.0741
Step 700: Train Loss 1.9671, Val Loss 1.9717, Val Perplexity: 7.1825
Step 800: Train Loss 1.8855, Val Loss 1.8904, Val Perplexity: 6.6221
Step 900: Train Loss 1.8308, Val Loss 1.8366, Val Perplexity: 6.2749
Step 1000: Train Loss 1.7850, Val Loss 1.7877, Val Perplexity: 5.9757
Step 1100: Train Loss 1.7423, Val Loss 1.7463, Val Perplexity: 5.7335
Step 1200: Train Loss 1.7125, Val Loss 1.7190, Val Perplexity: 5.5787
Step 1300: Train Loss 1.6798, Val Loss 1.6883, Val Perplexity: 5.4102
Step 1400: Train Loss 1.6619

In [ ]:
!nvidia-smi


Sun Jan 12 12:18:53 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              69W /  70W |   1011MiB / 15360MiB |     73%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--